## Vectorization of resulst

1. run inference (DONE - other notebook)
2. vectorize (DONE - this notebook)

#### to be done in this notebook: 
3. select random sample from inference (n = depending on how much we need to achieve accuracy)
4. plot over satellite image
5. ask user input
6. store user input

#### to be done in other notebook:

7. get pixel values 
8. retrain model, inputs:
    - existing weights and 
    - new pixel data from user input

repeat 

In [1]:
# add helper function location
import sys 
sys.path.append('../../functions')

# import helper functions 
from helpers import *

In [2]:
from skimage import filters, morphology
from rasterio import features 
from shapely.geometry import shape
from shapely import geometry
import itertools

import numpy as np
import cv2 as cv

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import geopandas as gpd

In [3]:
# functions


def array_to_linestrings(array, mask, affine):
    
    if np.all(mask) == True:
        final_lines = []
    else:
        # Convert the array to polygon geometries
        polygon_generator = features.shapes(array.astype('uint8'), mask=array, transform=affine)
        # Extract out the individual polygons, fixing any invald geometries using buffer(0)
        polygons = [shape(g).buffer(0) for g,v in polygon_generator]

        # Next, build up a binary array representing the outer boundary of the input array
        boundary_array = np.ones(array.shape)    

        # Convert the boundary to a polygon too
        boundary_generator = features.shapes(boundary_array.astype('uint8'), transform=affine)
        boundary = [shape(g) for g,v in boundary_generator][0]

        # Convert the mask to a polygon too
        if mask is not None:
            
            mask_generator = features.shapes(mask.astype('uint8'), 
                                             mask= mask > 0,
                                             transform=affine)
            mask_boundary = [shape(g).buffer(0) for g,v in mask_generator][0]

        # Convert the array polygons to linestrings
        exterior_rings = [g.exterior for g in polygons]
        interior_rings = list(itertools.chain.from_iterable([[i for i in g.interiors] for g in polygons if len(g.interiors) > 0]))
        all_rings =  exterior_rings + interior_rings 

        # Erase both the image boundary and the mask boundary from the extracted Linestrings
        erased_lines = all_rings #[g.difference(boundary.exterior).difference(mask_boundary.exterior) for g in all_rings]
        singlepart_lines = [list(g) if type(g) == geometry.multilinestring.MultiLineString else [g] for g in erased_lines ]
        final_lines = list(itertools.chain(*singlepart_lines))

    return final_lines 

# load classification raster

In [4]:
# load classified rasters 

import rasterio 
import glob
import os

from pprint import pprint

from shapely.ops import cascaded_union
import pandas as pd


name = ''

# find files in classification folder
raster_files_path = '../../data/nir_green_blue/classifications/{}*.tif'.format(name)
vectors_path = '../../data/nir_green_blue/vectors/'


raster_files = glob.glob(raster_files_path)
file_path = raster_files[1]

print(len(raster_files))
print('-------------------------------------')
pprint(raster_files[0:3])
print('-------------------------------------')
print(file_path[0:3])



502
-------------------------------------
['../../data/nir_green_blue/classifications/Jakarta_180_pansharpen_True_2018-04-03_03:22_103001007C082C00_05-18-2020_08:55:26.tif',
 '../../data/nir_green_blue/classifications/Jakarta_123_pansharpen_True_2019-08-11_03:17_103001009861DE00_05-18-2020_08:55:26.tif',
 '../../data/nir_green_blue/classifications/Medan_296_pansharpen_True_2020-03-03_04:05_10300100A367E500_05-18-2020_08:55:26.tif']
-------------------------------------
../


In [5]:
# create classification directory if it does not exist
make_dir(vectors_path)

Directory  ../../data/nir_green_blue/vectors/  already exists...


In [6]:
# set parameters:

verbose = 0


In [22]:
import random

geometry_list = []

for file_path in tqdm(raster_files):
    
    base_name = file_path.split('/')[-1].split('.')[0]

    try:
        data = rasterio.open(file_path).read()
    except Exception as e:
        print(e)
        print('skipping...')
        continue
    dataset = rasterio.open(file_path)

    affine = dataset.meta['transform']
    width = dataset.meta['width']
    height = dataset.meta['height']

    thresh = filters.threshold_otsu(data[0])
    thresh = 0.50
   

    data_otsu_treshold = data[0] > thresh

    water_data = data_otsu_treshold.astype('uint8')

    # rescale values to 0 - 255 to use image processing algorithms
    OldRange = (water_data.max() - water_data.min())  
    NewRange = (0 - 255)  
    NewValue = (((water_data - water_data.min()) * NewRange) / OldRange) + 0

    water_data_1 = NewValue.round().astype('uint8')

    water_blurred = cv.medianBlur(water_data_1,3)

    # Specify the minimum feature size in square meters, and then use info from the image metadata 
    # to translate to grid cell count.

    # Set the minimum feature size to 400 sq km (400,000 m)
    min_feature_size_m2 = 70.

    # From the image metadata, we can determine the area of a single grid cell
    try: 
        cell_height_m = image.rda.metadata['image']['groundSampleDistanceMeters']
    except: 
        cell_height_m = 0.40

    cell_area_m2 = cell_height_m**2

    # Finally, use the cell size to convert the minimum feature size to grid cells
    min_feature_size_cells = np.round((min_feature_size_m2/cell_area_m2)).astype('int64')



    # scikit-image modules for image science operators
    from skimage import filters, morphology

    # First, remove the small holes
    water_cleaned_1 = morphology.remove_small_holes(water_blurred, min_feature_size_cells)

    # Then remove the small objects
    water_cleaned_2 = morphology.remove_small_objects(water_cleaned_1, min_feature_size_cells, connectivity=2)
    
    if sum(sum(water_cleaned_2)) != 0:
        water_lines = array_to_linestrings(array = water_cleaned_2, mask = water_cleaned_2, affine = affine)

        # Run a basic simplification algorithm to smooth the lines
        water_lines_smooth = [g.simplify(0.001) for g in water_lines]

        # GLue it all together
        detection = cascaded_union(water_lines)

        import shapely.ops as ops
        # setu_smooth = setu_wgs.simplify(0.00003)
        detection_poly = ops.polygonize_full(detection)

        detection_poly = cascaded_union(detection_poly)

        buffer = 0# 1e-05

        detection_poly_smooth = detection_poly.buffer(-buffer).buffer(buffer)

        detection_poly_smooth

        detection_poly = ops.polygonize_full(detection_poly_smooth)[0]

        
        if verbose > 0: 
            for polygon_loop in detection_poly:
                x,y = polygon_loop.exterior.xy

                plt.plot(x,y,'r')
                plt.axis('equal')

            plt.show()
            
            # What is the minimum feature size in square meters and grid cells?
            print(min_feature_size_m2, 'sq. m')
            print(min_feature_size_cells, 'grid cells')
            
            print('source file: ', file_path)
            print('written to: ', geojson_file_path)

            print('---------------------------------------------------------')

        

        sample = random.randint(0,len(detection_poly)) - 1 


        geometry_list.append(detection_poly)

        geojson_file_path = vectors_path + '{}.geojson'.format(base_name)
    
        df = pd.DataFrame(shape(detection_poly), columns = ['geometry'])
        gdf = gpd.GeoDataFrame(df)
        gdf.to_file(geojson_file_path, driver='GeoJSON')


/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/morphology/misc.py:194: UserWarning: Any labeled images will be returned as a boolean array. Did you mean to use a boolean array?
  "Did you mean to use a boolean array?", UserWarning)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/morphology/misc.py:122: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:32: RuntimeWarning: invalid value encountered in true_divide


# upload results to S3

In [130]:
import logging
import boto3
from botocore.exceptions import ClientError
import os


def upload_file(file_name, bucket, prefix, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = prefix + '/' + os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name, )
    except ClientError as e:
        logging.error(e)
        return False
    return True



vector_list = glob.glob(vectors_path + '*.geojson')

In [131]:
'results/' + os.path.basename(file_name)

'results/kepler_test_csv_2020_11_06_14:37:36.csv'

In [132]:

dest_bucket = 'gcw-rimbunio-common-input-data-dev'
prefix = 'results'



for file in tqdm(vector_list):
    
    upload_file(file, dest_bucket, prefix)

# write shape to file

In [133]:
import geopandas
from datetime import datetime
from datetime import timedelta

import numpy as np
import pandas as pd

import json

from pprint import pprint

import namegenerator

In [134]:
def create_random_time_list(lenght, datetime):
    
    green_value_list = []
    date_time_green_list = []
        
    for i in range(lenght):

        new_time = date_time + timedelta(days = i/24)
        date_time_green_list.append(new_time.strftime("%Y/%m/%d %H:%M:%S"))
        green_value_list.append(np.round(1.5 * i + np.random.normal(0,1,1)[0] * 2,2))
        
    json_list_timestamps = json.dumps(date_time_green_list)
    json_list_green_over_time = json.dumps(green_value_list)
    
    
    
    return date_time_green_list, json_list_green_over_time

In [26]:
geojson_poly = geopandas.GeoSeries([detection_poly]).__geo_interface__

geojson_poly

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'GeometryCollection',
    'geometries': [{'type': 'Polygon',
      'coordinates': (((106.97074152081925, -6.096323626137261),
        (106.97075988229967, -6.096323626137261),
        (106.97075988229967, -6.096328216507366),
        (106.97076447266977, -6.096328216507366),
        (106.97076447266977, -6.096369529838303),
        (106.97075988229967, -6.096369529838303),
        (106.97075988229967, -6.096378710578512),
        (106.97075529192956, -6.096378710578512),
        (106.97075529192956, -6.096415433539345),
        (106.97075070155945, -6.096415433539345),
        (106.97075070155945, -6.096447566130075),
        (106.97074611118936, -6.096447566130075),
        (106.97074611118936, -6.096456746870284),
        (106.97074152081925, -6.096456746870284),
        (106.97074152081925, -6.096465927610492),
        (106.97073693044915, -6.096465927610492),
 

In [137]:
iets = detection_poly[0]

In [106]:
df =  pd.DataFrame(columns = ['id', 
                              'longitude',
                              'latitude',
                              '_geojson',
                              'name',
                              'time_analysis', 
                              'time_stamps',
                              'water_area_over_time',
                              'green_over_time',
                              'sediment_load_over_time',
                              'parcel_overlap' ])

now = datetime.now() # current date and time

# 1967/08/01 10:33:50.47

date_time = now
print("date and time:",date_time)

green_over_time_json_str = []
from shapely.geometry import Polygon




i_all = 0 

for geometry in geometry_list[0]:
    
    x,y = Polygon(geometry).representative_point().xy
    
    str_list_coords_coords = np.array(geometry.boundary.xy).T.tolist()

    str_list_coords = '{"type":"Feature","geometry":{"type":"Polygon","coordinates":[%s]}}' % str_list_coords_coords
    
    # print(geometry)
    
    name_waterbody = namegenerator.gen(separator = ' ')
    
    time_stamps, json_list_water_area_over_time = [None, None]
    time_stamps, json_list_green_over_time = [None, None]
    time_stamps, json_list_sediment_load_over_time = [None, None]
    
    json_list_parcel_overlap = json.dumps({'hak_milik' :None, 'none':None})
      
    df.loc[i_all] = [i_all, 
                     x[0],
                     y[0],
                     str_list_coords,
                     name_waterbody,
                     date_time.strftime("%Y/%m/%d %H:%M:%S"), 
                     time_stamps,
                     json_list_water_area_over_time,
                     json_list_green_over_time,
                     json_list_sediment_load_over_time,
                     json_list_parcel_overlap]
    
    i_all = i_all + 1

date and time: 2020-11-06 14:35:42.524173


In [107]:
df

,id,longitude,latitude,_geojson,name,time_analysis,time_stamps,water_area_over_time,green_over_time,sediment_load_over_time,parcel_overlap
0,0,106.742602,-6.136507,"{""type"":""Feature"",""geometry"":{""type"":""Polygon""...",ugly indigo spoonbill,2020/11/06 14:35:42,None,None,None,None,"{""hak_milik"": null, ""none"": null}"
1,1,106.741708,-6.135570,"{""type"":""Feature"",""geometry"":{""type"":""Polygon""...",gimpy magnolia hummingbird,2020/11/06 14:35:42,None,None,None,None,"{""hak_milik"": null, ""none"": null}"
2,2,106.740374,-6.136378,"{""type"":""Feature"",""geometry"":{""type"":""Polygon""...",gummy cyan jellyfish,2020/11/06 14:35:42,None,None,None,None,"{""hak_milik"": null, ""none"": null}"


### write kepler.gl proof csv data to file

In [111]:
now = datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
file_location = '../../data/kepler_test_data/'

make_dir(file_location)

file_name = 'kepler_test_csv_%s.csv' % now

df.to_csv(file_location + file_name, index = False)

print('file written to: ', file_location)

Directory  ../../data/kepler_test_data/  Created
file written to:  ../../data/kepler_test_data/


## Upload using the API

In [109]:
import requests

url = "http://ubuntu@ec2-54-196-69-114.compute-1.amazonaws.com:8080/api/v1/masterData"

payload = {
    'tagName': 'Test Data {}'.format(now),
    'config': '{}',
    'label': 'Test Master Data'
}

files = [
  ('file', open(file_location + file_name,'rb'))

]

headers= {
    'content-type': "application/json",
    'cache-control': "no-cache",
    'postman-token': "219161aa-1ca3-ffbe-6b61-29e757e686d6"
}

response = requests.request("POST", url, headers = headers, data = payload, files = files)


In [110]:
pprint(response.json())

{'error': {'expected': 61176,
           'length': 61176,
           'limit': 15360,
           'message': 'request entity too large',
           'status': 413,
           'statusCode': 413,
           'type': 'entity.too.large'},
 'message': 'request entity too large',
 'stack': 'PayloadTooLargeError: request entity too large\n'
          '    at readStream '
          '(/home/ubuntu/Rimbun-Backend/rimbun.io-back/backend/node_modules/raw-body/index.js:155:17)\n'
          '    at getRawBody '
          '(/home/ubuntu/Rimbun-Backend/rimbun.io-back/backend/node_modules/raw-body/index.js:108:12)\n'
          '    at read '
          '(/home/ubuntu/Rimbun-Backend/rimbun.io-back/backend/node_modules/body-parser/lib/read.js:77:3)\n'
          '    at jsonParser '
          '(/home/ubuntu/Rimbun-Backend/rimbun.io-back/backend/node_modules/body-parser/lib/types/json.js:135:5)\n'
          '    at Layer.handle [as handle_request] '
          '(/home/ubuntu/Rimbun-Backend/rimbun.io-back/backend